In [1]:
from project_example import *

In [2]:
data = load_data("active1000")

In [3]:
R1, R2 = AnalyzeUser(data, 1)

We recommend you these documents 

4867 with similarity: 0.7071
11628 with similarity: 0.7071
807 with similarity: 0.6667
5525 with similarity: 0.6667
2086 with similarity: 0.6030
1433 with similarity: 0.5976
247 with similarity: 0.5941
16685 with similarity: 0.5774
3001 with similarity: 0.5595
473 with similarity: 0.5536
478 with similarity: 0.5382
45 with similarity: 0.5241
5343 with similarity: 0.5156
2042 with similarity: 0.5116
1261 with similarity: 0.5014
5529 with similarity: 0.5000
8029 with similarity: 0.5000
8042 with similarity: 0.5000
11362 with similarity: 0.5000
2848 with similarity: 0.5000
17523 with similarity: 0.5000
17613 with similarity: 0.5000
7418 with similarity: 0.5000
6539 with similarity: 0.5000
8302 with similarity: 0.5000
8300 with similarity: 0.5000
6551 with similarity: 0.5000
14799 with similarity: 0.5000
16286 with similarity: 0.5000
15855 with similarity: 0.5000


In [4]:
docuTitle = data[['documentId', 'title', 'time']].copy()
docuTitle = docuTitle[~docuTitle['documentId'].isnull()]
docuTitle.drop_duplicates(subset=['documentId'], inplace=True)

In [5]:
docuTitle.insert(0, 'DocuID', range(0, 20344))

In [6]:
users = data[['userId']].copy()
users = users[~users['userId'].isnull()]
users.drop_duplicates(subset=['userId'], inplace=True)

In [7]:
users.insert(0, 'UserID', range(0, 1000))

In [8]:
event_logs = data[['userId', 'documentId', 'activeTime']].copy()

In [9]:
event_logs = event_logs[~event_logs['activeTime'].isnull()]

In [10]:
event_logs = event_logs[~event_logs['documentId'].isnull()]

In [11]:
column_names_to_normalize = ['activeTime']
x = event_logs[column_names_to_normalize].values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = event_logs.index)
event_logs[column_names_to_normalize] = df_temp

In [12]:
collabDocu = pd.merge(docuTitle, event_logs, on='documentId')
collabUserDocu = pd.merge(users, collabDocu, on='userId')

In [13]:
missing_pivot = collabUserDocu.pivot_table(values = 'activeTime', index = 'UserID', columns = 'DocuID')

In [14]:
read = {}
rows_indexes = {}
for i,row in missing_pivot.iterrows():
    rows = [x for x in range(0,len(missing_pivot.columns))]
    combine = list(zip(row.index, row.values, rows))
    readd = [(x,z) for x,y,z in combine if str(y) != 'nan']
    index = [i[1] for i in readd]
    row_names = [i[0] for i in readd]
    rows_indexes[i] = index
    read[i] = row_names

In [15]:
pivot_table = collabUserDocu.pivot_table(values = 'activeTime', index = 'UserID', columns = 'DocuID').fillna(0)

In [16]:
pivot_table = pivot_table.apply(np.sign)

In [17]:
notread = {}
notread_indexes = {}
for i,row in pivot_table.iterrows():
    rows = [x for x in range(0,len(missing_pivot.columns))]
    combine = list(zip(row.index, row.values, row))
    idx_row = [(idx,col) for idx, val, col in combine if not val > 0]
    indices = [i[1] for i in idx_row]
    row_names = [i[0] for i in idx_row]
    notread_indexes[i] = indices
    notread[i] = row_names

In [18]:
n = 5
cosine_knn = NearestNeighbors(n_neighbors = n, algorithm = 'brute', metric = 'cosine')
docu_cosine_knn_fit = cosine_knn.fit(pivot_table.T.values)
docu_distances, docu_indices = docu_cosine_knn_fit.kneighbors(pivot_table.T.values)

In [19]:
docus_dic = {}
for i in range(len(pivot_table.T.index)):
    docu_idx = docu_indices[i]
    col_names = pivot_table.T.index[docu_idx].tolist()
    docus_dic[pivot_table.T.index[i]] = col_names

In [20]:
topRecs = {}
for k,v in rows_indexes.items():
    docu_idx = [j for i in docu_indices[v] for j in i]
    docu_dist = [j for i in docu_distances[v] for j in i]
    combine = list(zip(docu_dist, docu_idx))
    diction = {i:d for d,i in combine if i not in v}
    zipped = list(zip(diction.keys(),diction.values()))
    sort = sorted(zipped, key = lambda x: x[1])
    recommendations = [(pivot_table.columns[i], d) for i,d in sort]
    topRecs[k] = recommendations

In [21]:
getrecommendations(1, pivot_table, topRecs)

We recommend you these documents 

4867 with similarity: 0.7071
11628 with similarity: 0.7071
807 with similarity: 0.6667
5525 with similarity: 0.6667
2086 with similarity: 0.6030
1433 with similarity: 0.5976
247 with similarity: 0.5941
16685 with similarity: 0.5774
3001 with similarity: 0.5595
473 with similarity: 0.5536
478 with similarity: 0.5382
45 with similarity: 0.5241
5343 with similarity: 0.5156
2042 with similarity: 0.5116
1261 with similarity: 0.5014
5529 with similarity: 0.5000
8029 with similarity: 0.5000
8042 with similarity: 0.5000
11362 with similarity: 0.5000
2848 with similarity: 0.5000
17523 with similarity: 0.5000
17613 with similarity: 0.5000
7418 with similarity: 0.5000
6539 with similarity: 0.5000
8302 with similarity: 0.5000
8300 with similarity: 0.5000
6551 with similarity: 0.5000
14799 with similarity: 0.5000
16286 with similarity: 0.5000
15855 with similarity: 0.5000


In [22]:
docu_distances = 1 - docu_distances
predictions = docu_distances.T.dot(pivot_table.T.values) / np.array([np.abs(docu_distances.T).sum(axis = 1)]).T
ground_truth = pivot_table.T.values[docu_distances.argsort()[0]]

In [23]:
accuracy = rmse(predictions,ground_truth)
print(accuracy)

0.9722903372235782


In [24]:
print(R1)
print(R2)

None
0.9722903372235782
